In [27]:
import inspect
from functools import wraps


def get_default_args(func):
    signature = inspect.signature(func)
    return {
        k: v.default
        for k, v in signature.parameters.items()
        if v.default is not inspect.Parameter.empty
    }


def block(func):
    @wraps(func)
    def filter_kwargs(*args, **kwargs):
        # FIXME: make this assert more nice
        # replace with printing info that we accept only named variables
        assert len(args) == 0
        print(func.__name__, func.__code__.co_varnames)
        default_args = get_default_args(func)
        print(kwargs)
        selected_kwargs = {}
        for key in func.__code__.co_varnames:
            # FIXME: make this assert more nice
            # e. g., display a message that graph doesn't compile
            assert (key in kwargs) or (key in default_args)
            if key in kwargs:
                selected_kwargs[key] = kwargs[key]
            else:
                selected_kwargs[key] = default_args[key]
        for key in kwargs:
            # TODO: make this configurable
            assert key in func.__code__.co_varnames
        result = func(**selected_kwargs)
        # FIXME: make this assert more nice
        # e. g., display a message that graph doesn't compile
        assert type(result) is dict
        # TODO: register function, inputs and outputs
        # TODO: how to call a fuction by name
        # https://stackoverflow.com/questions/3061/calling-a-function-of-a-module-by-using-its-name-a-string
        return result
    return filter_kwargs


@block
def f(a, b=4):
    return {'sum': a + b}

# f(a=2, c=3)
# f(a=2, b=3)


f ('a', 'b')
{'a': 2, 'b': 3}
5
